Función de las curvas ROC

In [ ]:
def grafico_roc(modelo, titulo):
    
    # Se calcula la predicción en términos de probabilidad
    y_probabilidad = modelo.predict_proba(X_test)
    
    # Se realiza el gráfico 
    skplt.metrics.plot_roc(y_test, y_probabilidad, figsize=(12,7))
    plt.title(titulo);

Función del accuracy

In [1]:
def accuracy(modelo):
    '''
    Solo toma como arumento el modelo del que se quiere
    calcular el accuracy. Es necesario que los objetos X_test e y_test 
    estén definidos previamente
    
    '''
    
    return modelo.score(X_test, y_test)

Función de la matriz de confusión

In [2]:
def plot_matriz_confusion(modelo, titulo):
    prediccion = modelo.predict(X_test) # la predicción
    
    confusion = confusion_matrix(y_test, prediccion) # la matriz de confusión
    
    # Definimos los elementos que van a ir dentro del gráfico: texto y número
    etiquetas = ['Verdaderos negativos', 'Falsos positivos', 'Falsos negativos', 'Verdaderos positivos']
    
    totales = ['{0:0.0f}'.format(value) for value in
                confusion.flatten()] # matriz 'plana'
    
    porcentuales = ['{0:.2%}'.format(value) for value in
                     confusion.flatten()/np.sum(confusion)] # calcula el porcentaje 
    
    labels = [f'{v1}\n{v2}\n{v3}' for v1, v2, v3 in
          zip(etiquetas, totales, porcentuales)]
    
    labels = np.asarray(labels).reshape(2,2)
    
    # Y ahora el plot
    plt.subplots(figsize=(12, 7))
    
    sns.heatmap(confusion, annot=labels, fmt='', cmap='Greens')
    plt.title(titulo)
    plt.xlabel('Valor real')
    plt.ylabel('Valor estimado')

Grafico de la roc con el punto optimo

In [6]:
def roc_punto_optimo (modelo, clase_positiva, nombre_modelo):
        '''
        Toma como argumentos el modelo que va a usar para los cálculos y el 
        nombre de la clase positiva. Necesario que los objetos X_test e y_test estén definidos. Hay que especificar el
        nombre del modleo, para la etiqueta del plot
        '''
        
        probabilidad_pagador = modelo.predict_proba(X_test)[:, 1] # calcula la prediccion en terminos de probabilidad
        
        # Valores necesarios para el cálculo
        tasa_falsos_positivos, tasa_verdaderos_positivos, umbrales = roc_curve(y_test, 
                                                                           probabilidad_pagador, 
                                                                           pos_label=clase_positiva)
        
        # Media geometrica y extraccion del optimo
        media_geometrica = np.sqrt(tasa_verdaderos_positivos * (1-tasa_falsos_positivos))
        indice_maxima_gmean = np.argmax(media_geometrica)

        # El plot 
        plt.figure(figsize=(12, 7))



        plt.plot([0,1], [0,1], # es simplemente una linea recta en diagonal 
                 linestyle='--', # linea punteada
                 color='red',
                 label='Modelo baseline') # etiqueta de la leyenda

        # Plot de la roc
        plt.plot(tasa_falsos_positivos, # en el eje x, los falsos positivos
                 tasa_verdaderos_positivos, # en el eje y, los verdaderos positivos
                 marker='.',
                 linewidth=0.1,
                 color='darkblue',
                 label=nombre_modelo)

        # Plot del punto óptimo
        plt.scatter(tasa_falsos_positivos[indice_maxima_gmean], # coordenada en el eje x
                    tasa_verdaderos_positivos[indice_maxima_gmean], # coordenada en el eje y
                    s=180, # tamaño
                    marker='o', 
                    color='orange', 
                    label='Óptimo')

        # Etiquetas 
        plt.title('Curva ROC y punto óptimo')
        plt.xlabel('Tasa de falsos positivos')
        plt.ylabel('Tasa de verdaderos positivos')
        plt.legend();      
        
        print(f' El umbral óptimo es {round(media_geometrica[indice_maxima_gmean], 2)}')

Precision-recall con punto optimo

In [5]:
def precision_recall_punto_optimo(modelo, clase_positiva, nombre_modelo): 
    '''
    Devuelve el gráfico de precision-recall con el punto óptimo. Necesario determinar la clase positva y 
    el nombre del modelo. Neceario que los objetos X_test e y_test estén definidos
    
    '''
    probabilidad_pagador = modelo.predict_proba(X_test)[:, 1] # calcula la prediccion en terminos de probabilida
    
    # Se obtienen los valores de la curva 
    precision, recall, umbral = precision_recall_curve(y_test, 
                                                       probabilidad_pagador, 
                                                       pos_label=clase_positiva)
    
    
    
    # Cálculo de la Fscore
    fscore = (2 * precision * recall) / (precision + recall)

    indice_fscore = np.argmax(fscore)

    plt.figure(figsize=(12, 7))

    # Calculamos los valores del modelo base
    modelo_base = len(y_test[y_test == clase_positiva].dropna()) / len(y_test)
    
    # Plot del modelo base
    plt.plot([0, 1], 
             [modelo_base, modelo_base], 
             linestyle='--',
             color='red',
             label='Modelo base')
    
    # Plot de los valores del XGBoost
    plt.plot(recall, # en el eje x los valores de recall
             precision,  # en el eje y los valores de precisión
             marker='.',
             color='darkblue',
             label=nombre_modelo)
    
    # Plot del punto óptimo
    plt.scatter(recall[indice_fscore], # mismo valor en cada eje
                precision[indice_fscore], 
                s=180, 
                marker='o', 
                color='orange', 
                label='Óptimo')
    
    # Etiquetas de los ejes
    plt.title('Curva Recall-Precisión')
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.legend();
    
    print(f' El nivel de probabilidad que optimiza la F1  es {round(fscore[indice_fscore], 2)}')